In [1]:
pip install boruta

     -------------------------------------- 56.6/56.6 kB 736.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn import preprocessing
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('UCI_Credit_Card.csv')
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [4]:
X = df.drop('ID',axis=1)
y = df['ID']

In [5]:
XX = np.array(X)
yy = np.array(y)

In [6]:
from sklearn.ensemble import RandomForestClassifier

# define random forest classifier
forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
forest.fit(XX, yy)

RandomForestClassifier(class_weight='balanced', max_depth=5, n_jobs=-1)

In [7]:
from boruta import BorutaPy

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(XX, yy)
# check selected features
feat_selector.support_
# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(XX)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	2
Rejected: 	22
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	2
Rejected: 	22
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	2
Rejected: 	22
Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	2
Rejected: 	22
Iteration: 	12 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	24


BorutaPy finished running.

Iteration: 	13 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	24


In [8]:
# iterating the columns
for col in df.columns:
    print(col)

ID
LIMIT_BAL
SEX
EDUCATION
MARRIAGE
AGE
PAY_0
PAY_2
PAY_3
PAY_4
PAY_5
PAY_6
BILL_AMT1
BILL_AMT2
BILL_AMT3
BILL_AMT4
BILL_AMT5
BILL_AMT6
PAY_AMT1
PAY_AMT2
PAY_AMT3
PAY_AMT4
PAY_AMT5
PAY_AMT6
default.payment.next.month


In [10]:
# zip my names, ranks, and decisions in a single iterable
feature_ranks = list(zip(df.keys(), feat_selector.ranking_, feat_selector.support_))

# iterate through and print out the results
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

Feature: ID                        Rank: 11,  Keep: False
Feature: LIMIT_BAL                 Rank: 24,  Keep: False
Feature: SEX                       Rank: 22,  Keep: False
Feature: EDUCATION                 Rank: 23,  Keep: False
Feature: MARRIAGE                  Rank: 8,  Keep: False
Feature: AGE                       Rank: 20,  Keep: False
Feature: PAY_0                     Rank: 16,  Keep: False
Feature: PAY_2                     Rank: 17,  Keep: False
Feature: PAY_3                     Rank: 20,  Keep: False
Feature: PAY_4                     Rank: 19,  Keep: False
Feature: PAY_5                     Rank: 18,  Keep: False
Feature: PAY_6                     Rank: 7,  Keep: False
Feature: BILL_AMT1                 Rank: 2,  Keep: False
Feature: BILL_AMT2                 Rank: 5,  Keep: False
Feature: BILL_AMT3                 Rank: 3,  Keep: False
Feature: BILL_AMT4                 Rank: 6,  Keep: False
Feature: BILL_AMT5                 Rank: 4,  Keep: False
Feature: BILL_AMT6   